# Predikcija ponašanja klijenata banke


## Sadržaj


1.  [Priprema okruženja](#priprema-okruzenja)
2.  [Priprema podataka](#priprema-podataka)
    1.  [Spljoštenje *golih* podataka](#spljostenje-golih-podataka)
    2.  [Dodavanje, izbacivanje i mijenjanje značajki](#dodavanje-izbacivanje-i-mijenjanje-znacajki),
3.  [Podjela](#podjela)
4.  [Konstrukcija modela](#konstrukcija-modela)
5.  [Inspekcija modela](#inspekcija-modela)
6.  [Predikcija](#predikcija)


## Priprema okruženja <a class="anchor" id="priprema-okruzenja"></a>


In [1]:
# Stadardna Python biblioteka
import functools
import sys

# Ostali paketi
import catboost as cb
import hyperopt as ho
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.core.arrays.categorical import Categorical
from pandas.core.frame import DataFrame
from pandas.core.indexes.base import Index
from pandas.core.series import Series

# Postavi prikaz grafova unutar biljeznice.
%matplotlib inline

# Postavi stil grafova na `ggplot'.
plt.style.use('ggplot')

# Ispisi inacice koristenog softvera.
print('Python version: {0}'.format(sys.version))
print('Numpy version: {0}'.format(np.__version__))
print('Pandas version: {0}'.format(pd.__version__))
print('Matplotlib version: {0}'.format(mpl.__version__))
print('CatBoost version: {0}'.format(cb.__version__))
print('hyperopt version: {0}'.format(ho.__version__))


Python version: 3.7.3 (default, Apr  3 2019, 05:39:12) 
[GCC 8.3.0]
Numpy version: 1.16.3
Pandas version: 0.24.2
Matplotlib version: 3.1.0
CatBoost version: 0.14.2
hyperopt version: 0.2


Portabilnosti radi, tablice ponekad spremamo u *.csv* formatu, ali, brzine radi, preferirat ćemo *.pkl* format. Stoga konstruirajmo funkciju koja *pametno* učitava tablicu &mdash; ako je *.pkl* datoteka dostupna, nju će učitati, a inače će iz *.csv* datoteke čitati tablicu sa zadanom interpretacijom datumskih i kategoričkih stupaca (kao i redaka zaglavlja i indeksa).


In [2]:
def ucitaj (ime, zaglavlje = 0, indeks = 0, datumi = None, kategorije = None):
    df = None

    try:
        df = pd.read_pickle('{0:s}.pkl'.format(ime))
    except FileNotFoundError:
        df = pd.read_csv(
            '{0:s}.csv'.format(ime),
            header = zaglavlje,
            index_col = indeks,
            parse_dates = datumi,
            infer_datetime_format = True if datumi is not None else None
        )
        for stupac in iter(kategorije if kategorije is not None else tuple()):
            vrijednosti = df[stupac].dropna().unique()
            if isinstance(vrijednosti, Categorical):
                vrijednosti = np.asarray(vrijednosti)
            try:
                if isinstance(vrijednosti, np.ndarray):
                    vrijednosti.sort()
                else:
                    vrijednosti = vrijednosti.sort_values()
            except (TypeError, ValueError, AttributeError):
                pass
            df[stupac] = df[stupac].astype(
                pd.api.types.CategoricalDtype(
                    categories = vrijednosti,
                    ordered = False
                )
            )
        df.to_pickle('{0:s}.pkl'.format(ime))

    return df


## Priprema podataka <a class="anchor" id="priprema-podataka"></a>


### Spljoštenje *golih* podataka <a class="anchor" id="spljostenje-golih-podataka"></a>


In [3]:
from script.pljoska import *


Za učitavanje treninškog *dataseta* koristi se poziv

```Python
df = ucitaj(
    'data/training_dataset_enc',
    zaglavlje = 0,
    indeks = 0,
    datumi = [
        'DATUM_IZVJESTAVANJA',
        'DATUM_OTVARANJA',
        'PLANIRANI_DATUM_ZATVARANJA',
        'DATUM_ZATVARANJA'
    ],
    kategorije = [
        'KLIJENT_ID',
        'OZNAKA_PARTIJE',
        'VALUTA',
        'VRSTA_KLIJENTA',
        'PROIZVOD',
        'VRSTA_PROIZVODA',
        'TIP_KAMATE'
    ]
)

df.PRIJEVREMENI_RASKID = df.PRIJEVREMENI_RASKID.map({'N' : 0, 'Y' : 1}).astype(int)

df.sort_values(
    by = [
        'DATUM_IZVJESTAVANJA',
        'DATUM_OTVARANJA',
        'STAROST'
    ],
    ascending = True,
    inplace = True
)
pojednostavi_indeks(df)

spljosteni_df = spljosti(df)
kategoriziraj(spljosteni_df)

```

Osim toga, u funkciji `spljosti` (iz skripte *pljsoka.py*) treba izmijeniti varijablu `znacajke` tako da se izbaci

```Python
('instance_id', functools.partial(bas_zadnji, stupac = 'instance_id'))

```

i da se doda

```Python
('DATUM_ZATVARANJA', functools.partial(feature_engineering.lastie, stupac = 'DATUM_ZATVARANJA'))

```

Također, funkciju `sredi_prijevremeni_raskid` (isto iz skripte *pljoska.py*) treba izmijeniti na sljedeći način

```Python
def sredi_prijevremeni_raskid (df):
    interval = pd.Timedelta(10, 'D')

    df.PRIJEVREMENI_RASKID = (
        df.DATUM_ZATVARANJA + interval < df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA
    ).astype(int)

    return df

```

Za učitavanje evaluacijskog *dataseta* koristi se poziv

```Python
df = ucitaj(
    'data/eval_dataset_nan',
    zaglavlje = 0,
    indeks = None,
    datumi = [
        'DATUM_OTVARANJA',
        'PLANIRANI_DATUM_ZATVARANJA'
    ],
    kategorije = [
        'KLIJENT_ID',
        'OZNAKA_PARTIJE',
        'VALUTA',
        'VRSTA_KLIJENTA',
        'PROIZVOD',
        'VRSTA_PROIZVODA',
        'TIP_KAMATE'
    ]
)

df.sort_values(
    by = [
        'OZNAKA_PARTIJE',
        'DATUM_OTVARANJA',
        'STAROST'
    ],
    ascending = True,
    inplace = True
)
pojednostavi_indeks(df)

spljosteni_df = spljosti(df)
kategoriziraj(spljosteni_df)

df.sort_values(by = ['instance_id'], ascending = True, inplace = True)
pojednostavi_indeks(df)

```


In [4]:
df = ucitaj(
    'data/validation_dataset_nan',
    zaglavlje = 0,
    indeks = None,
    datumi = [
        'DATUM_OTVARANJA',
        'PLANIRANI_DATUM_ZATVARANJA'
    ],
    kategorije = [
        'KLIJENT_ID',
        'OZNAKA_PARTIJE',
        'VALUTA',
        'VRSTA_KLIJENTA',
        'PROIZVOD',
        'VRSTA_PROIZVODA',
        'TIP_KAMATE'
    ]
)

spljosteni_df = None

try:
    spljosteni_df = ucitaj(
        'data/validation_dataset_flat',
        zaglavlje = 0,
        indeks = 0,
        datumi = [
            'PRVI_DATUM_OTVARANJA',
            'ZADNJI_DATUM_OTVARANJA',
            'PRVI_PLANIRANI_DATUM_ZATVARANJA',
            'ZADNJI_PLANIRANI_DATUM_ZATVARANJA'
        ],
        kategorije = [
            'KLIJENT_ID',
            'OZNAKA_PARTIJE',
            'PRVA_VALUTA',
            'ZADNJA_VALUTA',
            'VRSTA_KLIJENTA',
            'PROIZVOD',
            'VRSTA_PROIZVODA',
            'PRVI_TIP_KAMATE',
            'ZADNJI_TIP_KAMATE'
        ]
    )
except FileNotFoundError:
    df.sort_values(
        by = [
            'OZNAKA_PARTIJE',
            'DATUM_OTVARANJA',
            'STAROST'
        ],
        ascending = True,
        inplace = True
    )
    pojednostavi_indeks(df)

    spljosteni_df = spljosti(df)
    kategoriziraj(spljosteni_df)

    df.sort_values(by = ['instance_id'], ascending = True, inplace = True)
    pojednostavi_indeks(df)

    spljosteni_df.to_csv('data/validation_dataset_flat.csv')


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84879 entries, 0 to 84878
Data columns (total 14 columns):
instance_id                   84879 non-null int64
KLIJENT_ID                    84879 non-null category
OZNAKA_PARTIJE                84879 non-null category
DATUM_OTVARANJA               84879 non-null datetime64[ns]
PLANIRANI_DATUM_ZATVARANJA    84879 non-null datetime64[ns]
UGOVORENI_IZNOS               84879 non-null float64
VALUTA                        84879 non-null category
VRSTA_KLIJENTA                84879 non-null category
PROIZVOD                      84879 non-null category
VRSTA_PROIZVODA               84879 non-null category
VISINA_KAMATE                 83588 non-null float64
TIP_KAMATE                    84879 non-null category
STAROST                       84879 non-null int64
PRIJEVREMENI_RASKID           0 non-null float64
dtypes: category(7), datetime64[ns](2), float64(3), int64(2)
memory usage: 6.7 MB


In [6]:
spljosteni_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 75704 entries, 0 to 75703
Data columns (total 22 columns):
instance_id                          75704 non-null int64
KLIJENT_ID                           75704 non-null category
OZNAKA_PARTIJE                       75704 non-null category
PRVI_DATUM_OTVARANJA                 75704 non-null datetime64[ns]
ZADNJI_DATUM_OTVARANJA               75704 non-null datetime64[ns]
PRVI_PLANIRANI_DATUM_ZATVARANJA      75704 non-null datetime64[ns]
ZADNJI_PLANIRANI_DATUM_ZATVARANJA    75704 non-null datetime64[ns]
VRSTA_KLIJENTA                       75704 non-null category
PRVA_STAROST                         75704 non-null int64
STAROST                              75704 non-null int64
ZADNJA_STAROST                       75704 non-null int64
VRSTA_PROIZVODA                      75704 non-null category
PROIZVOD                             75704 non-null category
PRVI_UGOVORENI_IZNOS                 75704 non-null float64
ZADNJI_UGOVORENI_IZNOS    

### Dodavanje, izbacivanje i mijenjanje značajki <a class="anchor" id="dodavanje-izbacivanje-i-mijenjanje-znacajki"></a>


Pretpostavimo da nedostatak visine kamate možemo smatrati kao da je ona $ 0.00 \, \% $.


In [7]:
def ispuni_nedefinirane (df):
    df.PRVA_VISINA_KAMATE.fillna(0, inplace = True)
    df.ZADNJA_VISINA_KAMATE.fillna(0, inplace = True)

    return df


In [8]:
from script.ubaci_nove import *


In [9]:
def izbacivanje (df):
    stupci_za_izbaciti = [
        'KLIJENT_ID',
        'OZNAKA_PARTIJE',
        'PRVA_STAROST',
        'STAROST',
        'ZADNJA_STAROST',
        'PRVA_VALUTA',
        'PRVI_TIP_KAMATE',
        'PRVA_VISINA_KAMATE'
    ]

    df.drop(columns = stupci_za_izbaciti, errors = 'ignore', inplace = True)

    return df


Neke makroekonomske podatke i statistički izračunate vrijednosti pročitajmo iz spremljenih datoteka.


In [10]:
ekonomski_indikatori = ucitaj('data/ekonomski_indikatori', 0, 0, None, None)

score_vk = ucitaj('data/score_vrsta_klijenta', 0, 0, None, None)
score_p  = ucitaj('data/score_proizvod', 0, 0, None, None)


In [11]:
def pripremi (df):
    return izbacivanje(
        dodavanje(
            ekonomija(ispuni_nedefinirane(df), ekonomski_indikatori),
            ekonomski_indikatori,
            score_vk,
            score_p
        )
    )


Nakon poziva

```Python
pripremi(spljosteni_df)

```

tablica `spljosteni_df` sadrzavat će stupce

    PRVI_DATUM_OTVARANJA                 datetime64[ns]
    ZADNJI_DATUM_OTVARANJA               datetime64[ns]
    PRVI_PLANIRANI_DATUM_ZATVARANJA      datetime64[ns]
    ZADNJI_PLANIRANI_DATUM_ZATVARANJA    datetime64[ns]
    VRSTA_KLIJENTA                       category
    VRSTA_PROIZVODA                      category
    PROIZVOD                             category
    PRVI_UGOVORENI_IZNOS                 float64
    ZADNJI_UGOVORENI_IZNOS               float64
    ZADNJA_VALUTA                        category
    ZADNJI_TIP_KAMATE                    category
    ZADNJA_VISINA_KAMATE                 float64
    TREND_bdp                            float64
    TREND_inflacija                      float64
    TREND_nezaposlenosti                 float64
    TREND_nafta                          float64
    STD_bdp                              float64
    STD_inflacija                        float64
    STD_nezaposlenosti                   float64
    STD_nafta                            float64
    PROSJEK_bdp                          float64
    PROSJEK_inflacija                    float64
    PROSJEK_nezaposlenosti               float64
    duljina_dani                         int64
    duljina_godine                       float64
    slozeni_kamatni                      float64
    slozeni_kamatni_po_danu              float64
    slozeni_kamatni_po_godini            float64
    jednostavni_kamatni                  float64
    jednostavni_kamatni_po_danu          float64
    jednostavni_kamatni_po_godini        float64
    visina_kamate_kvadrat                float64
    broj_godina_u_krizi                  int64
    krizne_puta_ukupno                   float64
    veci_krizni_dug                      float64
    optimisticne_godine                  int64
    mean_probit_kam                      float64
    mean_score_kam                       float64
    promjena_tipa_kamate                 int64
    promjena_visine_kamate               int64
    promjena_ugovorenog_iznosa           int64
    PRIJEVREMENI_RASKID                  int64

Ako je `spljosteni_df` dobiven spljoštenjem treninškog *dataseta*, sadržavat će još i stupac

    DATUM_ZATVARANJA                     datetime64[ns]

a, ako je dobiven spljoštenjem evaluacijskog *dataseta*, sadržavat će stupac

    instance_id                          int64

Stupci `PROSJEK_...` i `STD_...` predstavljaju proječnu vrijednost i standardnu devijaciju vrijednosti kroz vrijeme (planiranog, ugovorenog) trajanja ugovora respektivno, a `TREND_...` koeficijent smjera linearne regresije.

Složeni kamatni račun je, na žalost, pogrešno izračunat jer se za visinu kamate ne uzima ta vrijednost kao postotak (ne dijeli se sa $ 100 $), nego se ona uzima takva kakva jest &mdash; složeni kamatni račun je računat po formuli
$$ \text{složeni kamatni račun} = \left( 1 + \text{prva visina kamate} \right)^{\text{duljina (godine)}} \cdot \text{prvi ugovoreni iznos} \text{.} $$
Jednostavni kamatni račun je, doduše, točno izračunat formulom
$$ \text{jednostavni kamatni račun} = \left( 1 + \text{duljina (godine)} \cdot \frac{\text{zadnja visina kamate}}{100} \right) \cdot \text{zadnji ugovoreni iznos} \text{.} $$
Kamatni računi po danu odnosno po godini izračunati su jednostavno dijeljenjem dobivene vrijednosti kamatnog računa s trajanjem ugovora u odgovarajućoj mjernoj jedinici.

Veći krizni dug izračunat je po formuli
$$ 1.5 \cdot \text{broj godina u krizi} \cdot \text{jednostavni kamatni račun} \text{,} $$
a optimistične godine kao $ 1 $ ako je zadnji planirani datum zatvaranja unutar $ 2 $ godine od kraja krize (moguće je da će klijent tada otvaranjem novog kredita, s boljim uvjetima, zatvoriti postojeći kredit od kojeg nije preostalo mnogo za vratiti) i $ 0 $ inače.

Značajke `mean_score_kam` i `mean_probit_kam` posebno su zanimljive. Za svaku kategorijsku vrijednost stupca `PROIZVOD` i za svaku kategorijsku vrijednost stupca `VRSTA_KLIJENTA` prvo su (posebno) izračunati statistički izračunate *vjerojatnosti* da će ugovor s tom vrijednosti biti prijevremeno raskinut (točnije, izračunati su udjeli prijevremeno raskinutih ugovora), i to tako da su te vjerojatosti posebno izračunate unutar grupe vrste proizvoda *A* i grupe vrste proizvoda *L*. Zatim su, ponovo unutar svake od grupa vrste proizvoda, izračunati lokalni maksimumi udjela prijevremeno raskinutih ugovora po razredima visine kamate (razredi su dobiveni pozivom funkcije `matplotlib.pyplot.hist`), a tada se za svaku visinu kamate uzima njezina udaljenost od sredine razreda s maksimalnom vjerojatnosti prijevremenog raskida kojemu ta visina kamate *teži* (za vrstu proizvoda *A* su, na primjer, $ 3 $ lokalna maksimuma po visini kamate pa se ovisno o visini kamate računa udaljenost od nekog od ta $ 3 $ lokalna maksimuma). Konačna vrijednost značajke `mean_probit_kam` izračunata je po formuli
$$ \text{mean_probit_kam} = \log \left( k \left( \text{vrsta proizvoda} , \text{visina kamate} \right) \cdot \exp \left( \frac{ f \left( \text{vrsta proizvoda} , p_{\text{vrsta klijenta}} \right) + f \left( \text{vrsta proizvoda} , p_{\text{proizvod}} \right)}{2} \right) \right) \text{,} $$
gdje je $ k \left( \text{vrsta proizvoda} , \text{visina kamate} \right) $ recipročna vrijednost udaljenosti od odgovarajućeg lokalnog maksimuma visine kamate ako je vrsta proizvoda *A*, a inače udaljenost od odgovarajućeg lokalnog maksimuma visine kamate, i
$$ f \left( \text{vrsta proizvoda} , p \right) = \begin{cases} \Phi^{{- 1}} \left( p \right) & \text{vrsta proizvoda} = A \\ \Phi^{{- 1}} \left( 1 - p \right) & \text{inače} \end{cases} \text{,} $$
gdje je $ \Phi $ funkcija distribucije normalne slučajne varijable s očekivanjem $ 0 $ i varijancom $ 1 $. Ako je $ p $ za manje od $ \varepsilon $ udaljen od $ 0 $ ili $ 1 $, uzima se $ p = \varepsilon $ odnosno $ p = 1 - \varepsilon $ (u kodu se koristi $ \varepsilon = 10^{{- 6}} $). Za vrstu proizvoda *A* vrijednost značajke `mean_probit_kam` bit će *visoka* ako je po svim kriterijima (vrsta klijenta, proizvod, visina kamate) vrlo vjerojatno da će ugovor biti prijevremeno raskinut, a vrlo niska (po apsolutnoj vrijednosti velika, ali inače strogo manja od $ 0 $) ako je vrlo vjerojatno da ugovor ne će biti prijevremeno raskinut. Za vrstu proizvoda *L* raspored je obrnut. Značajka `mean_score_kam` računa se po sličnoj formuli, ali u definiciji funkcije $ f $ umjesto $ \Phi^{{- 1}} $ uzima se identiteta.


In [12]:
try:
    spljosteni_df = ucitaj(
        'data/validation_dataset_ready',
        zaglavlje = 0,
        indeks = 0,
        datumi = [
            'PRVI_DATUM_OTVARANJA',
            'ZADNJI_DATUM_OTVARANJA',
            'PRVI_PLANIRANI_DATUM_ZATVARANJA',
            'ZADNJI_PLANIRANI_DATUM_ZATVARANJA'
        ],
        kategorije = [
            'ZADNJA_VALUTA',
            'VRSTA_KLIJENTA',
            'PROIZVOD',
            'VRSTA_PROIZVODA',
            'ZADNJI_TIP_KAMATE'
        ]
    )
except FileNotFoundError:
    pripremi(spljosteni_df)

    spljosteni_df.to_csv('data/validation_dataset_ready.csv')


In [13]:
spljosteni_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 75704 entries, 0 to 75703
Data columns (total 45 columns):
instance_id                          75704 non-null int64
PRVI_DATUM_OTVARANJA                 75704 non-null datetime64[ns]
ZADNJI_DATUM_OTVARANJA               75704 non-null datetime64[ns]
PRVI_PLANIRANI_DATUM_ZATVARANJA      75704 non-null datetime64[ns]
ZADNJI_PLANIRANI_DATUM_ZATVARANJA    75704 non-null datetime64[ns]
VRSTA_KLIJENTA                       75704 non-null category
VRSTA_PROIZVODA                      75704 non-null category
PROIZVOD                             75704 non-null category
PRVI_UGOVORENI_IZNOS                 75704 non-null float64
ZADNJI_UGOVORENI_IZNOS               75704 non-null float64
ZADNJA_VALUTA                        75704 non-null category
ZADNJI_TIP_KAMATE                    75704 non-null category
ZADNJA_VISINA_KAMATE                 75704 non-null float64
PRIJEVREMENI_RASKID                  75704 non-null int64
TREND_bdp             

Nakon što vrijednosti datuma više nisu potrebne kao vrijednosti datuma, numerificiramo ih pozivom funkcije `pandas.to_numeric` i dijeljenjem s $ 10^{12} $.


In [14]:
def numerificiraj_datume (df):
    df.PRVI_DATUM_OTVARANJA = pd.to_numeric(
        df.PRVI_DATUM_OTVARANJA
    ) / 1.0e+12
    df.ZADNJI_DATUM_OTVARANJA = pd.to_numeric(
        df.ZADNJI_DATUM_OTVARANJA
    ) / 1.0e+12
    df.PRVI_PLANIRANI_DATUM_ZATVARANJA = pd.to_numeric(
        df.PRVI_PLANIRANI_DATUM_ZATVARANJA
    ) / 1.0e+12
    df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA = pd.to_numeric(
        df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA
    ) / 1.0e+12

    return df


## Podjela <a class="anchor" id="podjela"></a>


Za predikciju su konstruirana $ 3 $ modela:

1.  model za predikciju ugovora vrste proizvoda *A* zadnjeg planiranog datuma zatvaranja do $ 6 $. listopada $ 2016 $. godine,
2.  model za predikciju ugovora vrste proizvoda *L* zadnjeg planiranog datuma zatvaranja do $ 6 $. listopada $ 2016 $. godine,
3.  model za predikciju ugovora zadnjeg planiranog datuma zatvaranja nakon $ 6 $. listopada $ 2016 $. godine.

Ove posljednje zovemo *zeznutima* iz sljedećih razloga:

1.  vjerojatno su ekonomske prilike do $ 2016 $. godine, kao godine nakon zadnje velike krize, danas dovoljno jasne i iskristalizirane da su njihovi utjecaji dobro predvidljivi (osim određenih predikcija od $ 2019 $. godine nadalje, za neke makroekonoske pokazatelje ni nemamo dovoljno podataka za računanje prosječnih vrijednsoti, standardne devijacije i trendova),
2.  planirani datum zatvaranja nakon zadnjeg datuma izvještavanja ($ 31 $. prosinca $ 2018 $. godine) mogu stvarati probleme: čak i ako bi po nekim pokazateljima ugovor vjerojatno bio prijevremeno raskinut, taj prijevremeni raskid možda se još nije ni dogodio jer je ugovor ugovoren do, na primjer, $ 2040 $. godine &mdash; u tom slučaju model mora predviđati vrijednost u bazi podataka banke, a ne stvarnu vjerojatnost hoće li ugovor biti prijevremeno raskinut (što bi se možda desilo tek $ 2030 $. godine, ali ne još).


In [15]:
def podijeli (df):
    granica = pd.Timestamp('2019-10-06')

    df_zez = df.loc[df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA > granica].copy()
    df = df.drop(index = df_zez.index).copy()

    df.reset_index(drop = True, inplace = True)
    df_zez.reset_index(drop = True, inplace = True)

    po_vrsti_proizvoda = df.groupby('VRSTA_PROIZVODA')
    po_vrsti_proizvoda_zez = df_zez.groupby('VRSTA_PROIZVODA')

    df_A = po_vrsti_proizvoda.get_group('A').copy()
    df_L = po_vrsti_proizvoda.get_group('L').copy()
    df_A_zez = po_vrsti_proizvoda_zez.get_group('A').copy()
    df_L_zez = po_vrsti_proizvoda_zez.get_group('L').copy()

    del df
    del df_zez

    df_A.reset_index(drop = True, inplace = True)
    df_L.reset_index(drop = True, inplace = True)
    df_A_zez.reset_index(drop = True, inplace = True)
    df_L_zez.reset_index(drop = True, inplace = True)

    df_A.drop(columns = ['VRSTA_PROIZVODA'])
    df_L.drop(columns = ['VRSTA_PROIZVODA'])

    return (df_A, df_L, df_A_zez, df_L_zez)


Tablicu dijelimo pozivom

```Python
df_A, df_L, df_A_zez, df_L_zez = podijeli(spljosteni_df)

```

Modeli nadalje *očekuju* numerificirane datume (funkcijom `numerificiraj_datume`), pa su nužni i pozivi

```Python
numerificiraj_datume(df_A)
numerificiraj_datume(df_L)
numerificiraj_datume(df_A_zez)
numerificiraj_datume(df_L_zez)

```


In [16]:
df_A, df_L, df_A_zez, df_L_zez = podijeli(spljosteni_df)

numerificiraj_datume(df_A)
numerificiraj_datume(df_L)
numerificiraj_datume(df_A_zez)
numerificiraj_datume(df_L_zez)


,instance_id,PRVI_DATUM_OTVARANJA,ZADNJI_DATUM_OTVARANJA,PRVI_PLANIRANI_DATUM_ZATVARANJA,ZADNJI_PLANIRANI_DATUM_ZATVARANJA,VRSTA_KLIJENTA,VRSTA_PROIZVODA,PROIZVOD,PRVI_UGOVORENI_IZNOS,ZADNJI_UGOVORENI_IZNOS,...,visina_kamate_kvadrat,broj_godina_u_krizi,krizne_puta_ukupno,veci_krizni_dug,optimisticne_godine,mean_probit_kam,mean_score_kam,promjena_ugovorenog_iznosa,promjena_tipa_kamate,promjena_visine_kamate
0,56492,1493251.2,1493251.2,1653955.2,1653955.2,1410,L,FD0100,32052.24,32052.24,...,0.1600,0,0.000000,0.000000e+00,0,1.299530,1.700566,1,1,1
1,56435,1364169.6,1364169.6,1682812.8,1682812.8,1550,L,FD0100,53111.34,53111.34,...,4.0000,3,30.291788,2.844990e+04,0,0.599849,1.001048,1,1,1
2,56812,1057017.6,1057017.6,1696032.0,1696032.0,1410,L,FD0100,6382.53,6382.53,...,1.0000,8,161.994264,4.548278e+03,0,1.090438,1.491474,1,1,1
3,56013,1521504.0,1521504.0,1588204.8,1588204.8,1410,L,FD0100,263850.13,263850.13,...,0.0000,0,0.000000,0.000000e+00,0,1.418011,1.819047,1,1,1
4,56814,1265587.2,1265587.2,1585612.8,1585612.8,1410,L,FD0100,28542.49,28542.49,...,4.0000,6,60.846412,3.046855e+04,0,0.600073,1.001109,1,1,1
5,46734,1515715.2,1515715.2,1578873.6,1578873.6,1550,L,TM0109,150000.00,150000.00,...,2.2500,0,0.000000,0.000000e+00,0,1.622633,1.545643,1,1,1
6,55655,1428451.2,1428451.2,1590883.2,1590883.2,1550,L,FD0100,160747.25,160747.25,...,3.2400,1,5.147195,5.118528e+04,0,0.719037,1.120236,1,1,1
7,42971,1492819.2,1492819.2,1587600.0,1587600.0,1410,L,TM0109,126638.21,126638.21,...,0.1600,0,0.000000,0.000000e+00,0,2.047371,1.970218,1,1,1
8,56467,1411430.4,1411430.4,1730332.8,1730332.8,1410,L,FD0100,83851.38,83851.38,...,4.0000,2,20.210953,2.992394e+04,0,0.600073,1.001109,1,1,1
9,56561,1084147.2,1084147.2,1725062.4,1725062.4,1410,L,FD0100,3724.15,3724.15,...,1.0000,8,162.476129,2.647334e+03,0,1.090438,1.491474,1,1,1


## Konstrukcija modela <a class="anchor" id="konstrukcija-modela"></a>


Model konstruiramo kao objekt klase `catboost.CatBoostClassifier`, a njegove *hiperparametre* (argumente konstruktora) optimiziramo paketom *hyperopt*.

Funkcija za takvu optimizaciju može, na primjer, biti sljedeća:

```Python
def optimizacija (
    space,
    train_X,
    train_y,
    test_X,
    test_y,
    cat_stupci,
    loss
):
    if space['learning_rate'] < 0:
        return {'status' : ho.STATUS_FAIL}
    if space['border_count'] < 1 or space['border_count'] > 254:
        return {'status' : ho.STATUS_FAIL}

    train_pool = cb.Pool(train_X, train_y, cat_features = cat_stupci)
    test_pool = cb.Pool(test_X, test_y, cat_features = cat_stupci)

    model = cb.CatBoostClassifier(
        iterations = 600,
        learning_rate = space['learning_rate'],
        depth = 9,
        l2_leaf_reg = space['l2_leaf_reg'],
        border_count = space['border_count'],
        od_type = space['od_type'],
        leaf_estimation_method = space['leaf_estimation_method'],
        random_seed = np.maximum(np.abs(space['random_seed']), 1),
        random_strength = space['random_strength'],
        bagging_temperature = space['bagging_temperature'],
        task_type ='GPU',
        sampling_unit = space['sampling_unit'],
    )

    model.fit(train_pool, eval_set = test_pool, verbose = False, plot = False)

    return {'loss' : 1 - loss(model, test_X, test_y), 'status' : ho.STATUS_OK}

```

Varijabla `cat_stupci` lista je imena stupaca čije vrijednosti su kategoričkog tipa, a `train_X`, `train_y`, `test_X` i `test_y` vjerojatno su dovoljno jasne same po sebi &mdash; dobivene su podjelom originalnog treninškof *dataseta* na podskup koji služi za trening i podskup koji služi za evaluaciju treninga (naravno, stupac `DATUM_ZATVARANJA` je izbačen).

Pretpostavimo da model želimo optimizirati tako da maksimiziramo njegovu točnost. U tu svrhu definiramo funkciju `loss` koja vraća $ 1 - \text{točnost} \left( \text{model} \right) $.

```Python
def loss (model, X, y):
    return 1 - (model.predict(X) == y).astype(float).sum() / X.shape[0]

```

Osim ovoga, mogli smo konstruirati i funkcije koje vraćaju $ 1 - \text{preciznost} \left( \text{model} \right) $, $ 1 - \text{odziv} \left( \text{model} \right) $, $ 1 - F_{1} \left( \text{model} \right) $ ili nešto sasvim četvrto/peto. Bitno je da je vrijednost bliska $ 0 $ ako je model zadovoljavajuće dobar, a bliska $ 1 $ ako je loš.

Optimizacija hiperparametara u predstavljenom okruženju može se provesti sljedećim pozivima (najbolje vrijednosti pronađene u `max_evals` iteracija &mdash; v. na dnu `best = ho.fmin(...)` &mdash; *spremljene* su u objektu `best`):

```Python
space = {
    'learning_rate' : ho.hp.uniform('learning_rate', 0.09, 0.95),
    'l2_leaf_reg' : ho.hp.qlognormal('l2_leaf_reg', 1, 5, 0.5),
    'border_count': ho.hp.randint('border_count', 300),
    'od_type' : ho.hp.choice('od_type', ['IncToDec', 'Iter']),
    'leaf_estimation_method' : ho.hp.choice(
        'leaf_estimation_method',
        ['Newton', 'Gradient']
    ),
    'random_seed' : ho.hp.randint('random_seed', 1000),
    'random_strength' : ho.hp.lognormal('random_strength', 1, 1),
    'bagging_temperature' : ho.hp.lognormal('bagging_temperature', 1, 1),
    'sampling_unit' : ho.hp.choice('sampling_unit', ['Group', 'Object'])
}

trials = ho.Trials()

best = ho.fmin(
    fn = functools.partial(
        optimizacija,
        train_X = train_X,
        train_y = train_y,
        test_X = test_X,
        test_y = test_y,
        cat_stupci = cat_stupci,
        loss = loss
    ),
    space = space,
    algo = ho.tpe.suggest,
    max_evals = 12,
    trials = trials
)

```

Nakon provedene optimizacije, model se sada trenira pozivima

```Python
train_pool = cb.Pool(train_X, train_y, cat_features = cat_stupci)
test_pool = cb.Pool(test_X, test_y, cat_features = cat_stupci)

model = cb.CatBoostClassifier(
    iterations = 1000,
    learning_rate = best['learning_rate'],
    depth = 9,
    l2_leaf_reg = best['l2_leaf_reg'],
    border_count = best['border_count'],
    od_type = best['od_type'],
    leaf_estimation_method = best['leaf_estimation_method'],
    random_seed = np.maximum(np.abs(best['random_seed']), 1),
    random_strength = best['random_strength'],
    bagging_temperature = best['bagging_temperature'],
    task_type ='GPU',
    sampling_unit = best['sampling_unit'],
)

model.fit(
    train_pool,
    eval_set = test_pool,
    verbose = False,
    plot = True,
    early_stopping_rounds = 50
)

```

U stvarnom treniranju modela (ne u optimizaciji hiperparametara) broj iteracija povećali smo na $ 1000 $, ali smo postavili `early_stopping_rounds` na $ 50 $ &mdash; ako u $ 50 $ iteracija vrijednost *loss*-funkcije na testnom *datasetu* nije poboljšana, treniranje se prekida. Model koristi postavu iz one iteracije u kojoj je vrijednost *loss*-funkcije na testnom *datasetu* bila najniža, čak i ako `early_stopping_rounds` ne postavimo i dopustimo da se model *pretrenira* velikim brojem iteracija.

Konačno model možemo spremiti pozivom

```Python
model.save('istrenirani_model')

```

Argument metode `catboost.CatBoostClassifier.save` je lokacija datoteke u koju model želimo *spremiti*. Analogno tada model učitavamo iz datoteke pozivom

```Python
model.load('istrenirani_model')

```


## Inspekcija modela <a class="anchor" id="inspekcija-modela"></a>


In [17]:
def tezine (model):
    return pd.DataFrame(
        data = model.get_feature_importance(),
        index = model.feature_names_,
        columns = ['Tezine']
    ).sort_values(by = 'Tezine', ascending = False)


Učitajmo postojeće modele.


In [18]:
model_A = cb.CatBoostClassifier()
model_L = cb.CatBoostClassifier()
model_zez = cb.CatBoostClassifier()


In [19]:
model_A.load_model('model_A')
model_L.load_model('model_L')
model_zez.load_model('model_zez')


Provjerimo značajnosti značajki na temelju kojih modeli *odlučuju*. Svakoj značajki pridružena je vrijednost od $ 0 $ do $ 100 $, a zbroj tih vrijednosti iznosi $ 100 $ &mdash; ta vrijednost pokazuje udio (u postotku) značajnosti značajke. Time je rješenje, očitio, interpretabilno.


In [20]:
tezine(model_A)


,Tezine
ZADNJI_PLANIRANI_DATUM_ZATVARANJA,23.260158
PRVI_PLANIRANI_DATUM_ZATVARANJA,16.780593
promjena_visine_kamate,7.198764
ZADNJI_TIP_KAMATE,4.243602
PROIZVOD,3.710901
duljina_dani,3.552803
ZADNJI_DATUM_OTVARANJA,2.496980
krizne_puta_ukupno,2.411208
jednostavni_kamatni_po_danu,2.196662
promjena_tipa_kamate,2.160996


In [21]:
tezine(model_L)


,Tezine
duljina_dani,44.514573
broj_godina_u_krizi,30.819161
ZADNJI_PLANIRANI_DATUM_ZATVARANJA,20.810886
PRVI_PLANIRANI_DATUM_ZATVARANJA,3.211381
VRSTA_KLIJENTA,0.643999
PRVI_DATUM_OTVARANJA,0.000000
slozeni_kamatni_po_danu,0.000000
slozeni_kamatni_po_godini,0.000000
jednostavni_kamatni,0.000000
jednostavni_kamatni_po_danu,0.000000


In [22]:
tezine(model_zez)


,Tezine
promjena_visine_kamate,13.950717
ZADNJI_PLANIRANI_DATUM_ZATVARANJA,13.054419
PRVI_PLANIRANI_DATUM_ZATVARANJA,7.168370
PRVI_DATUM_OTVARANJA,6.561310
ZADNJI_TIP_KAMATE,5.035716
visina_kamate_kvadrat,3.755600
ZADNJI_DATUM_OTVARANJA,3.750640
PROIZVOD,3.713883
ZADNJA_VALUTA,3.669686
mean_score_kam,3.225462


## Predikcija <a class="anchor" id="predikcija"></a>


*Zeznute* ugovore možda nije dovoljno predviđati modelom za *zeznute* ugovore. Srećom, klasifikatori klase `catboost.CatBoostClassifier` mogu, osim konačne klasifikacije, vratiti vjerojatnost pripadnosti. Kombiniranjem vjerojatnosti dvaju (ili čak više, ali ovdje to nije potrebno jer nemamo *više* modela) modela konačnu klasifikaciju možemo sami izračunati.


In [23]:
def kombinacija (model_1, model_2, X, combo):
    return np.argmax(
        combo(
            model_1.predict_proba(X[model_1.feature_names_]),
            model_2.predict_proba(X[model_2.feature_names_])
        ),
        axis = 1
    )


Predikciju za *ne-zeznute* ugovore provodimo pozivom

```Python
df_A['PRIJEVREMENI_RASKID'] = model_A.predict(df_A[model_A.feature_names_])
df_L['PRIJEVREMENI_RASKID'] = model_L.predict(df_L[model_L.feature_names_])

```

Za predikciju *zeznutih* ugovora prvo definiramo neku kombinirajuću funkciju

```Python
combo_1 = lambda a, b : a
combo_2 = lambda a, b : b
combo_1_plus_2 = lambda a, b : a + b
combo_1_mul_2 = lambda a, b : a * b

```

Treća i četvrta funkcija su zapravo simplifikacija aritmetičke i geometrijske sredine bez težina (dijeljenje s $ 2 $ ili korijenovanje ne će utjecati na maksimume i minimume). Osim tih sredina, mogli smo definirati i neku treću sredinu, a mogli smo definirati i neku težinsku sredinu.

Pretpostavimo da smo varijable `combo_A_zez` i `combo_L_zez` fiksirali na neke od spomenutih funkcija (eksplicitno napisanih ili teoretski spomenutih). Sada predikciju zeznutih ugovora provodimo pozivima

```Python
df_A_zez['PRIJEVREMENI_RASKID'] = kombinacija(
    model_A,
    model_zez,
    df_A_zez,
    combo_A_zez
)
df_L_zez['PRIJEVREMENI_RASKID'] = kombinacija(
    model_L,
    model_zez,
    df_L_zez,
    combo_L_zez
)

```

Daljnji kod pojednostavimo reindeksiranjem podskupova *dataseta*

```Python
df_A.index = df_A.instance_id
df_L.index = df_L.instance_id
df_A_zez.index = df_A_zez.instance_id
df_L_zez.index = df_L_zez.instance_id

```

Uvrštavanje predikcije u originalni *dataset* dalje provodimo pozivima

```Python
df['PRIJEVREMENI_RASKID'] = 0

df.loc[df_A.index, 'PRIJEVREMENI_RASKID'] = df_A.PRIJEVREMENI_RASKID
df.loc[df_L.index, 'PRIJEVREMENI_RASKID'] = df_L.PRIJEVREMENI_RASKID
df.loc[df_A_zez.index, 'PRIJEVREMENI_RASKID'] = df_A_zez.PRIJEVREMENI_RASKID
df.loc[df_L_zez.index, 'PRIJEVREMENI_RASKID'] = df_L_zez.PRIJEVREMENI_RASKID

df.PRIJEVREMENI_RASKID = df.PRIJEVREMENI_RASKID.map({0 : 'N', 1 : 'Y'})

```


In [24]:
df['PRIJEVREMENI_RASKID'] = 0


In [25]:
df_A['PRIJEVREMENI_RASKID'] = model_A.predict(df_A[model_A.feature_names_])
df_L['PRIJEVREMENI_RASKID'] = model_L.predict(df_L[model_L.feature_names_])


In [26]:
combo_A_zez = lambda a, b : b
combo_L_zez = lambda a, b : 0.01 * a ** 1.5 + b ** 1.5


In [27]:
df_A_zez['PRIJEVREMENI_RASKID'] = kombinacija(
    model_A,
    model_zez,
    df_A_zez,
    combo_A_zez
)
df_L_zez['PRIJEVREMENI_RASKID'] = kombinacija(
    model_L,
    model_zez,
    df_L_zez,
    combo_L_zez
)


In [28]:
df_A.index = df_A.instance_id
df_L.index = df_L.instance_id
df_A_zez.index = df_A_zez.instance_id
df_L_zez.index = df_L_zez.instance_id


In [29]:
df.loc[df_A.index, 'PRIJEVREMENI_RASKID'] = df_A.PRIJEVREMENI_RASKID
df.loc[df_L.index, 'PRIJEVREMENI_RASKID'] = df_L.PRIJEVREMENI_RASKID
df.loc[df_A_zez.index, 'PRIJEVREMENI_RASKID'] = df_A_zez.PRIJEVREMENI_RASKID
df.loc[df_L_zez.index, 'PRIJEVREMENI_RASKID'] = df_L_zez.PRIJEVREMENI_RASKID


In [30]:
df.PRIJEVREMENI_RASKID.astype(float).sum() / df.shape[0]


0.48027191649288986

In [31]:
df.PRIJEVREMENI_RASKID = df.PRIJEVREMENI_RASKID.map({0 : 'N', 1 : 'Y'})


In [32]:
df.to_csv('student.csv')
